In [1]:
from qiskit_experiments.library import T1
from qiskit_experiments.library import StandardRB
from qiskit_experiments.framework import ParallelExperiment
import numpy as np
from qiskit_ibm_runtime.fake_provider import FakePerth
from qiskit_aer import AerSimulator
from qiskit.qobj.utils import MeasLevel

In [2]:
%run NeutralAtomProcessor.ipynb

In [4]:
lengths = np.arange(1, 500, 25)
num_samples = 10
seed = 1010
qubits = [0]

processor_1q = NeutralAtomProcessor(1)
pulse_backend_1q = QiskitPulseSimulator(processor_1q)

experiment_1q = StandardRB(qubits, lengths, num_samples=num_samples, seed=seed)
experiment_1q.set_transpile_options(optimization_level=1, basis_gates=["rx","rz","cz"])
experiment_1q._set_backend(pulse_backend_1q)
experiment_1q._finalize()
transpiled_circuits = experiment_1q._transpiled_circuits()

print(transpiled_circuits[0])

global phase: 3π/2
        ┌──────────┐┌───────┐ ░ ┌──────────┐┌───────┐ ░ ┌─┐
     q: ┤ Rz(-π/2) ├┤ Rx(π) ├─░─┤ Rz(-π/2) ├┤ Rx(π) ├─░─┤M├
        └──────────┘└───────┘ ░ └──────────┘└───────┘ ░ └╥┘
meas: 1/═════════════════════════════════════════════════╩═
                                                         0 


In [5]:
experiment_1q.analysis.set_options(outcome="0")
experiment_data_1q = experiment_1q._initialize_experiment_data()
run_opts = experiment_1q.run_options.__dict__
_ = [this_circ.remove_final_measurements() for this_circ in transpiled_circuits]
jobs_1q = [experiment_1q._run_jobs(this_circ, **run_opts)[0] for this_circ in transpiled_circuits]

for i, job in enumerate(jobs_1q):
    d = job.result().results[0].header.to_dict()
    d["metadata"] = transpiled_circuits[i].metadata
    job.result().results[0].header.from_dict(d)

Circuit has  4  gates after transpiling
Circuit complete
Circuit has  47  gates after transpiling
Circuit complete
Circuit has  93  gates after transpiling
Circuit complete
Circuit has  138  gates after transpiling
Circuit complete
Circuit has  186  gates after transpiling
Circuit complete
Circuit has  230  gates after transpiling
Circuit complete
Circuit has  276  gates after transpiling
Circuit complete
Circuit has  325  gates after transpiling
Circuit complete
Circuit has  362  gates after transpiling
Circuit complete
Circuit has  402  gates after transpiling
Circuit complete
Circuit has  448  gates after transpiling
Circuit complete
Circuit has  490  gates after transpiling
Circuit complete
Circuit has  535  gates after transpiling
Circuit complete
Circuit has  582  gates after transpiling
Circuit complete
Circuit has  627  gates after transpiling
Circuit complete
Circuit has  674  gates after transpiling
Circuit complete
Circuit has  727  gates after transpiling
Circuit complete
C

AttributeError: 'Qobj' object has no attribute 'states'

In [ ]:
experiment_data_1q.add_jobs(jobs_1q, timeout=None)
experiment_1q.analysis.set_options(outcome="0") # qutip−qip uses ”0” instead of ”00”
expdata_1q = experiment_1q.analysis.run(experiment_data_1q, gate_error_ratio=False)
results_1q = expdata_1q.analysis_results()
display(expdata_1q.figure(0))